In [1]:
import tensorflow as tf
from tensorflow.keras import optimizers 
from tensorflow.keras import backend as K 
from tensorflow.keras import regularizers 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense,Activation,Dropout,Flatten 
from tensorflow.keras.layers import Embedding,Conv1D,MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,Bidirectional
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from transformers import BertTokenizer, TFBertModel

import numpy as np
import os
import pandas as pd

2023-03-03 17:50:22.155913: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-03 17:50:28.004914: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-03 17:50:28.004978: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-03 17:50:36.685844: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU 
from keras.layers import CuDNNGRU, CuDNNLSTM, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from tensorflow.keras.layers import InputSpec, Layer
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping

In [4]:

!git clone https://huggingface.co/NepBERTa/NepBERTa

Cloning into 'NepBERTa'...
remote: Enumerating objects: 15, done.
remote: Total 15 (delta 0), reused 0 (delta 0), pack-reused 15
Unpacking objects: 100% (15/15), 146.21 KiB | 411.00 KiB/s, done.


In [3]:
def bert_tweets_model():
    bert_encoder = TFBertModel.from_pretrained('./NepBERTa')
    input_word_ids = tf.keras.Input(shape=(140,), dtype=tf.int32, name="input_ids")
    last_hidden_states = bert_encoder(input_word_ids)[0]    
    x = tf.keras.layers.Bidirectional(LSTM(64, dropout=0.4, recurrent_dropout=0.4))(last_hidden_states)
    x1 = tf.keras.layers.Dense(32,activation='relu')
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs=input_word_ids, outputs=output)
    
    return model

In [4]:
model = bert_tweets_model()

2023-03-03 17:51:10.987772: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-03-03 17:51:10.987821: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (proson-Predator-PH315-52): /proc/driver/nvidia/version does not exist
2023-03-03 17:51:11.135152: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-03 17:51:12.595814: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 93763584 exceeds 10% of free system memory.
2023-03-03 17:51:12.964520: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 93763584 exceeds 10% 

In [5]:
model.load_weights("model.h5")

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
vocab_file_dir = './NepBERTa/' 
tokenizer = BertTokenizer.from_pretrained(vocab_file_dir,
                                        strip_accents=False,
                                         clean_text=False )

In [ ]:
def bert_encode(data):
    tokens = tokenizer.batch_encode_plus(data, max_length=140, padding='max_length', truncation=True)
    
    return tf.constant(tokens['input_ids'])

In [ ]:
from translit import *

2023-03-03 14:09:08 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


हेल्लो


In [49]:
text = 'maile boksi dekhe'
transliteraed = engine.translit_sentence(text)
print(f'original:{text}')
print(f'Transliterated:{transliteraed}')
test_encoded = bert_encode([transliteraed])
dev_dataset = (
    tf.data.Dataset
    .from_tensor_slices((test_encoded))
    .batch(32)
)
predictions = model.predict(dev_dataset)
sentiment = ['Hate' if np.array((predictions>0.5).astype(int))[0][0]==1 else 'Not Hate']
print(sentiment)

original:maile boksi dekhe
Transliterated:माइले बोक्सी देखे
1/1 [==============================] - 0s 262ms/step
['Hate']
